# SBERT Hyperparameter Tuning Experiment 1

**Experiment focus:** Tuning learning rate and batch size for SBERT fine-tuning.

This notebook explores the effect of different learning rates and batch sizes on SBERT performance for scientific claim source retrieval.

In [3]:
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import torch
import numpy as np

/opt/anaconda3/envs/sbert-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading

Load the train and dev datasets, and prepare InputExample objects for SBERT.

In [4]:
# Load train and dev data
train_df = pd.read_csv('../subtask4b_query_tweets_train.tsv', sep='\t', names=['post_id', 'tweet_text', 'cord_uid'])
dev_df = pd.read_csv('../subtask4b_query_tweets_dev.tsv', sep='\t', names=['post_id', 'tweet_text', 'cord_uid'])
with open('../subtask4b_collection_data.pkl', 'rb') as f:
    papers_df = pickle.load(f)
papers_df['text'] = papers_df['title'] + '. ' + papers_df['abstract']
paper_dict = dict(zip(papers_df['cord_uid'], papers_df['text']))

# Prepare training examples
train_samples = []
for _, row in train_df.iterrows():
    if row['cord_uid'] in paper_dict:
        tweet = row['tweet_text']
        paper = paper_dict[row['cord_uid']]
        train_samples.append(InputExample(texts=[tweet, paper]))

# Prepare dev examples
dev_samples = []
for _, row in dev_df.iterrows():
    if row['cord_uid'] in paper_dict:
        tweet = row['tweet_text']
        paper = paper_dict[row['cord_uid']]
        dev_samples.append(InputExample(texts=[tweet, paper]))

## Hyperparameter Grid

Define the learning rates and batch sizes to try.

In [5]:
learning_rates = [2e-5, 3e-5, 5e-5]
batch_sizes = [8, 16, 32]
epochs = 4  # Fixed for this experiment
warmup_steps = 100  # Fixed for this experiment
model_name = 'multi-qa-mpnet-base-cos-v1'

## Training and Evaluation Loop

For each combination, fine-tune SBERT and evaluate on the dev set.

In [6]:
def evaluate_mrr(model, dev_df, papers_df, top_k=5):
    # Encode dev queries
    query_embeddings = model.encode(dev_df['tweet_text'].tolist(), show_progress_bar=True, convert_to_tensor=True)
    # Encode papers
    paper_embeddings = model.encode(papers_df['text'].tolist(), show_progress_bar=True, convert_to_tensor=True)
    # Compute similarity
    paper_norm = torch.nn.functional.normalize(paper_embeddings, p=2, dim=1)
    paper_ids = papers_df['cord_uid'].tolist()
    predictions = []
    for query_emb in query_embeddings:
        query_norm = torch.nn.functional.normalize(query_emb.unsqueeze(0), p=2, dim=1)
        similarity = torch.matmul(query_norm, paper_norm.T).squeeze()
        top_indices = torch.topk(similarity, k=min(top_k, len(paper_norm))).indices.tolist()
        preds = [paper_ids[i] for i in top_indices]
        predictions.append(preds)
    # Calculate MRR
    scores = []
    for i, row in dev_df.iterrows():
        gold = row['cord_uid']
        preds = predictions[i]
        if gold in preds:
            rank = preds.index(gold) + 1
            scores.append(1.0 / rank)
        else:
            scores.append(0.0)
    return np.mean(scores) if scores else 0.0

In [7]:
results = []
for lr in learning_rates:
    for batch_size in batch_sizes:
        print(f'Training {model_name} | lr={lr} | batch={batch_size}')
        model = SentenceTransformer(model_name)
        train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=batch_size)
        train_loss = losses.MultipleNegativesRankingLoss(model)
        model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            epochs=epochs,
            warmup_steps=warmup_steps,
            show_progress_bar=True,
            optimizer_params={'lr': lr}
        )
        mrr = evaluate_mrr(model, dev_df, papers_df, top_k=5)
        results.append({
            'model': model_name,
            'lr': lr,
            'batch_size': batch_size,
            'epochs': epochs,
            'warmup': warmup_steps,
            'dev_mrr': mrr
        })
        print(f'Result: MRR={mrr}')

Training multi-qa-mpnet-base-cos-v1 | lr=2e-05 | batch=8


Epoch:   0%|          | 0/4 [00:26<?, ?it/s]


KeyboardInterrupt: 

## Save Results

Save the results to a CSV file for later analysis.

In [ ]:
pd.DataFrame(results).to_csv('hyperparam_results_1.csv', index=False)
print('Results saved to hyperparam_results_1.csv')